# Importation des librairies

In [ ]:
import os
import random

import numpy as np

import tensorflow as tf
import tensorflow_datasets as tfds

from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.optimizers import Adam

from sklearn.metrics import confusion_matrix

import matplotlib.pyplot as plt
import seaborn as sns

# Graines

In [ ]:
seed = 42

os.environ['PYTHONHASHSEED'] = str(seed)
random.seed(seed)
np.random.seed(seed)
tf.random.set_seed(seed)

# Chargement des données

In [ ]:
X_train, y_train = tfds.as_numpy(tfds.load(
    'rock_paper_scissors',
    split='train',
    batch_size=-1,
    as_supervised=True,
))

In [ ]:
X_test, y_test = tfds.as_numpy(tfds.load(
    'rock_paper_scissors',
    split='test',
    batch_size=-1,
    as_supervised=True,
))

# Pré-traitement des données

In [ ]:
feature_vector_length = X_train.shape[1]*X_train.shape[2]*X_train.shape[3]
num_classes = 3

In [ ]:
X_train = np.reshape(X_train, (X_train.shape[0], feature_vector_length))

In [ ]:
X_test = np.reshape(X_test, (X_test.shape[0], feature_vector_length))

In [ ]:
one_hot_encoded_y_train = to_categorical(y_train, num_classes)

In [ ]:
one_hot_encoded_y_test = to_categorical(y_test, num_classes)

# Création du modèle

In [ ]:
# Set the input shape
input_shape = (feature_vector_length,)
print(f'Feature shape: {input_shape}')

# Create the model
model = Sequential()

model.add(Dense(1024, input_shape=input_shape, activation='relu'))
model.add(Dense(num_classes, activation='softmax'))

# Compilation du modèle

In [ ]:
# Configure the model
opt = Adam(learning_rate=0.001)

model.compile(loss='categorical_crossentropy', optimizer=opt, metrics=['accuracy'])

# Entrainement du modèle

In [ ]:
# Start training
history = model.fit(X_train, one_hot_encoded_y_train, epochs=50, batch_size=32, verbose=1, validation_split=0.2)

In [ ]:
plt.figure(figsize=(8, 5), dpi=100)
plt.title("Loss")
plt.plot(history.history['loss'], label="Train Dataset")
plt.plot(history.history['val_loss'], label="Validation Dataset")
plt.xlabel("Epoch")
plt.legend()
plt.show()

In [ ]:
plt.figure(figsize=(8, 5), dpi=100)
plt.title("Accuracy")
plt.plot(history.history['accuracy'], label="Train Dataset")
plt.plot(history.history['val_accuracy'], label="Validation Dataset")
plt.xlabel("Epoch")
plt.legend()
plt.show()

# Performances du modèle sur le jeu d'apprentissage

## Taux de classification

In [ ]:
model.evaluate(X_train, one_hot_encoded_y_train)

## Matrice de confusion

In [ ]:
y_predict_train = np.argmax(model.predict(X_train), axis=-1)

In [ ]:
cm = confusion_matrix(y_train, y_predict_train, normalize='true')

In [ ]:
counts = ["{0:0.0f}\n".format(value) for value in confusion_matrix(y_train, y_predict_train, normalize=None).flatten()]
percentages = ["{0:.2%}".format(value) for value in confusion_matrix(y_train, y_predict_train, normalize='true').flatten()]

box_labels = [f"{v1}{v2}" for v1, v2 in zip(counts, percentages)]
box_labels = np.asarray(box_labels).reshape(cm.shape[0],cm.shape[1])

plt.figure(figsize=(8,5), dpi=100)

plt.title("Matrice de confusion (train dataset)")

sns.heatmap(cm,
            vmin=0.0,
            vmax=1.0,
            cmap='Blues',
            annot=box_labels,
            fmt='',
            xticklabels=["Rock", "Paper", "Scissors"],
            yticklabels=["Rock", "Paper", "Scissors"])

plt.xlabel("Valeurs prédites")
plt.ylabel("Valeurs réelles")

plt.show()

# Performances du modèle sur le jeu de test

## Taux de classification

In [ ]:
model.evaluate(X_test, one_hot_encoded_y_test)

## Matrice de confusion

In [ ]:
y_predict_test = np.argmax(model.predict(X_test), axis=-1)

In [ ]:
cm = confusion_matrix(y_test, y_predict_test, normalize='true')

In [ ]:
counts = ["{0:0.0f}\n".format(value) for value in confusion_matrix(y_test, y_predict_test, normalize=None).flatten()]
percentages = ["{0:.2%}".format(value) for value in confusion_matrix(y_test, y_predict_test, normalize='true').flatten()]

box_labels = [f"{v1}{v2}" for v1, v2 in zip(counts, percentages)]
box_labels = np.asarray(box_labels).reshape(cm.shape[0],cm.shape[1])

plt.figure(figsize=(8,5), dpi=100)

plt.title("Matrice de confusion (test dataset)")

sns.heatmap(cm,
            vmin=0.0,
            vmax=1.0,
            cmap='Blues',
            annot=box_labels,
            fmt='',
            xticklabels=["Rock", "Paper", "Scissors"],
            yticklabels=["Rock", "Paper", "Scissors"])

plt.xlabel("Valeurs prédites")
plt.ylabel("Valeurs réelles")

plt.show()